In [20]:
import tensorflow as tf

import numpy as np
import pandas as pd
import os
import glob
import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

eps = np.finfo(float).eps

plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

import sys

from tensorflow.keras import Model, layers
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy
from tensorflow_privacy.privacy.optimizers.dp_optimizer import DPGradientDescentGaussianOptimizer

In [6]:
# load dataset
train, test = tf.keras.datasets.mnist.load_data()
train_data, train_labels = train
test_data, test_labels = test

train_data = np.array(train_data, dtype=np.float32) / 255
test_data = np.array(test_data, dtype=np.float32) / 255

train_data = train_data.reshape(train_data.shape[0], 28, 28, 1)
test_data = test_data.reshape(test_data.shape[0], 28, 28, 1)

train_labels = np.array(train_labels, dtype=np.int32)
test_labels = np.array(test_labels, dtype=np.int32)

train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=10)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=10)

*Privacy applied for SGD


- bounded sensitivity of each gradient: we need to limit how much each individual training point sampled in a minibatch can influence gradient computations and the resulting updates applied to model parameters. This can be done by clipping each gradient computed on each training point.

- Random noise is sampled and added to the clipped gradients to make it statistically impossible to know whether or not a particular data point was included in the training dataset by comparing the updates SGD applies when it operates with or without this particular data point in the training dataset.

In [18]:
epochs = 15
batch_size = 250

l2_norm_clip = 1.5 # gradient clip
noise_multiplier = 1.3 #random noise added
num_microbatches = 250
learning_rate = 0.25

assert batch_size % num_microbatches == 0.


In [21]:
class Classifier(Model):
    # Set layers.
    def __init__(self):
        super(Classifier, self).__init__()
        self.conv1 = layers.Conv2D(16, 8, strides=2, padding='same', activation='relu', input_shape=(28, 28, 1))
        self.maxpool1 = layers.MaxPool2D(2, 1)
        self.conv2 = layers.Conv2D(32, 4, strides=2, padding='valid', activation='relu')
        self.maxpool2 = layers.MaxPool2D(2, 1)
        self.flatten = layers.Flatten()

        # Fully connected layer.
        self.fc1 = layers.Dense(32, activation='relu')
        self.dropout = layers.Dropout(rate=0.5)
        self.out = layers.Dense(10)

    # Set forward pass.
    def call(self, x, is_training=False):
        x = tf.reshape(x, [-1, 28, 28, 1])
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.dropout(x, training=is_training)
        x = self.out(x)
        if not is_training:
            # tf cross entropy expect logits without softmax, so only
            # apply softmax when not training.
            x = tf.nn.softmax(x)
        return x
    


In [22]:
optimizer = DPGradientDescentGaussianOptimizer(
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier,
    num_microbatches=num_microbatches,
    learning_rate=learning_rate)

loss = tf.keras.losses.CategoricalCrossentropy(
    from_logits=True, reduction=tf.losses.Reduction.NONE)

In [23]:
model = Classifier()
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

model.fit(train_data, train_labels,
          epochs=epochs,
          validation_data=(test_data, test_labels),
          batch_size=batch_size)

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 4s 67us/sample - loss: 2.3023 - accuracy: 0.1166 - val_loss: 2.3022 - val_accuracy: 0.1497
Epoch 2/15
60000/60000 [==============================] - 4s 60us/sample - loss: 2.3015 - accuracy: 0.1143 - val_loss: 2.2995 - val_accuracy: 0.1209
Epoch 3/15
60000/60000 [==============================] - 4s 60us/sample - loss: 2.3012 - accuracy: 0.1187 - val_loss: 2.3034 - val_accuracy: 0.1015
Epoch 4/15
60000/60000 [==============================] - 4s 59us/sample - loss: 2.3048 - accuracy: 0.1043 - val_loss: 2.3022 - val_accuracy: 0.1160
Epoch 5/15
60000/60000 [==============================] - 4s 60us/sample - loss: 2.3049 - accuracy: 0.1060 - val_loss: 2.3074 - val_accuracy: 0.1062
Epoch 6/15
60000/60000 [==============================] - 4s 59us/sample - loss: 2.3075 - accuracy: 0.1020 - val_loss: 2.3057 - val_accuracy: 0.1222
Epoch 7/15
60000/60000 [==============================] 

In [24]:
compute_dp_sgd_privacy.compute_dp_sgd_privacy(n=60000, batch_size=250, noise_multiplier=1.3, epochs=15, delta=1e-5)

DP-SGD with sampling rate = 0.417% and noise_multiplier = 1.3 iterated over 3600 steps satisfies differential privacy with eps = 1.18 and delta = 1e-05.
The optimal RDP order is 17.0.


(1.1799006739827, 17.0)